In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import re
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

stop_words=set(stopwords.words('english'))
df = pd.read_csv('/home/met/Downloads/twitter-airline-sentiment/Tweets.csv')
st= PorterStemmer()
text= df.values[:, 10]
labels=df.iloc[:,1].values
stemmedWords=[]
pattern=r"http\S+"

def normalization(features):
    filtered_features = []
    for sent in features:
        sent= re.sub(pattern,"",sent)
        sent = word_tokenize(sent)
        tmpStr=""
        for word in sent:
            if word not in stop_words:
                tmpStr=tmpStr+" "+st.stem(word.lower())
        filtered_features.append(tmpStr)
    return filtered_features


target=[]
for label in labels:
    if label=="neutral":
        target.append(0)
    elif label=="positive":
        target.append(1)
    else:
        target.append(-1)

text= normalization(text)

X_train, X_test, y_train, y_test = train_test_split(text, target, test_size = 0.2, random_state = 101)


# print("text: ",text)
vectorizer = TfidfVectorizer()
X_train_tfidf=vectorizer.fit_transform(X_train)

X_test_tfidf = vectorizer.transform(X_test)


# print("target" ,target)



clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

predicted = clf.predict(X_test_tfidf)

print(precision_recall_fscore_support(y_test,predicted, average='micro'))
print("Score: ",clf.score(X_test_tfidf,y_test))
print("mean squared error ",mean_squared_error(y_test, predicted, multioutput='raw_values'))



neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_tfidf,y_train)
predicted=neigh.predict(X_test_tfidf)
print("K nearest neighbours: ",precision_recall_fscore_support(y_test,predicted, average='micro'))

clf = RandomForestClassifier(n_estimators=100, max_depth=2,
random_state=0)
clf.fit(X_train_tfidf, y_train)
predicted=clf.predict(X_test_tfidf)

print("mean squared error ",mean_squared_error(y_test, predicted, multioutput='raw_values'))
print(precision_recall_fscore_support(y_test,predicted, average='micro'))
